In [1]:
# Preparing the environment
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

In [2]:
import torch
import numpy as np
import smplx

from human_body_prior.tools.omni_tools import copy2cpu as c2c
from os import path as osp

support_dir = './data'

In [3]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
comp_device

device(type='cpu')

In [4]:
amass_npz_fname = osp.join(support_dir, 'YOGI_2_latest_smplx_neutral/train/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.npz') # the path to body data
bdata = np.load(amass_npz_fname)

# you can set the gender manually and if it differs from data's then contact or interpenetration issues might happen
subject_gender = bdata['gender']

print('Data keys available:%s'%list(bdata.keys()))

print('The subject of the mocap sequence is  {}.'.format(subject_gender))


Data keys available:['gender', 'surface_model_type', 'mocap_frame_rate', 'mocap_time_length', 'trans', 'poses', 'betas', 'num_betas', 'root_orient', 'pose_body', 'pose_jaw', 'pose_eye', 'pose_hand']
The subject of the mocap sequence is  neutral.


In [5]:
from human_body_prior.body_model.body_model import BodyModel

bm_fname = osp.join(support_dir, 'body_models/smplh/{}/model.npz'.format(subject_gender))

num_betas = 16 # number of body parameters

bm = BodyModel(f'{support_dir}/models_lockedhead/smplx/SMPLX_NEUTRAL.npz', 'smplx', num_betas=num_betas).to(comp_device)
faces = c2c(bm.f)

In [6]:
time_length = len(bdata['trans'])

body_parms = {
    'root_orient': torch.Tensor(bdata['poses'][:, :3]).to(comp_device), # controls the global root orientation
    'pose_body': torch.Tensor(bdata['poses'][:, 3:66]).to(comp_device), # controls the body
    'pose_hand': torch.Tensor(bdata['poses'][:, 66:]).to(comp_device), # controls the finger articulation
    'trans': torch.Tensor(bdata['trans']).to(comp_device), # controls the global body position
    'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=time_length, axis=0)).to(comp_device), # controls the body shape. Body shape is static
    # 'dmpls': torch.Tensor(bdata['dmpls'][:, :num_dmpls]).to(comp_device) # controls soft tissue dynamics
}

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in body_parms.items()])))
print('time_length = {}'.format(time_length))


Body parameter vector shapes: 
root_orient: torch.Size([892, 3]) 
pose_body: torch.Size([892, 63]) 
pose_hand: torch.Size([892, 99]) 
trans: torch.Size([892, 3]) 
betas: torch.Size([892, 16])
time_length = 892


In [7]:
mosh = f"{support_dir}/mosh/train/Boat_Pose/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.pkl"

In [8]:
import pickle as pkl
from moyo.eval.com_evaluation import smplx_to_mesh, visualize_mesh
from moyo.utils.constants import frame_select_dict_combined as frame_select_dict
import trimesh

# all_pose_names = [pp_name.replace('_stageii', '') for pp_name in pp_names]
selected_frame = frame_select_dict['_'.join(mosh.split('_')[5:-1])]

pp_pkl = mosh
pp_params = pkl.load(open(pp_pkl, 'rb'))
# read height, weight, gender meta
height = None
weight = None
gender = 'neutral'
subj_id = 'unknown'
pp_body_model_output, smplx_params, transls, faces = smplx_to_mesh(pp_params, f'{support_dir}/models_lockedhead/smplx/SMPLX_NEUTRAL.npz','smplx', gender=gender)

pp_mesh = visualize_mesh(pp_body_model_output, faces, frame_id=880)
scene = trimesh.scene.scene.Scene(pp_mesh)
scene.set_camera(angles=(1.2,0,0), distance=3)
scene.show()

KeyError: '03596_Boat_Pose_or_Paripurna_Navasana_-a'

In [16]:
import pickle as pkl
from moyo.eval.com_evaluation import smplx_to_mesh, visualize_mesh
from moyo.utils.constants import frame_select_dict_combined as frame_select_dict
import trimesh

accad_npz = osp.join(support_dir, 'ACCAD/Male2MartialArtsKIcks_c3d/G2_-_Sidekick_leading_left_stageii.npz') 

pp_pkl = accad_npz
pp_params = np.load(accad_npz)
# read height, weight, gender meta
height = None
weight = None
gender = 'neutral'
subj_id = 'unknown'
pp_body_model_output, smplx_params, transls, faces = smplx_to_mesh(pp_params, f'{support_dir}/models_lockedhead/smplx/SMPLX_NEUTRAL.npz','smplx', gender=gender)

pp_mesh = visualize_mesh(pp_body_model_output, faces, frame_id=135)
scene = trimesh.scene.scene.Scene(pp_mesh)
scene.set_camera(angles=(1.2,0,0), distance=5)
scene.show()

ValueError: string is not a file: `./data/v_templates/220923_yogi_03596_minimal_simple_female/mesh.ply`

In [17]:
list(pp_params.keys())

['gender',
 'surface_model_type',
 'mocap_frame_rate',
 'mocap_time_length',
 'markers_latent',
 'latent_labels',
 'markers_latent_vids',
 'trans',
 'poses',
 'betas',
 'num_betas',
 'root_orient',
 'pose_body',
 'pose_hand',
 'pose_jaw',
 'pose_eye']

In [ ]:
print(f"Frame Rate: {pp_params['mocap_frame_rate']} Time Length: {pp_params['mocap_time_length']}, Total Frames = {pp_params['mocap_frame_rate'] * pp_params['mocap_time_length']}")

Frame Rate: 120.0 Time Length: 2.7916666666666665, Total Frames = 335.0


In [105]:
import os

support_dir = 'data'

data_folder = "mosh/train/"

for pose_file in os.listdir(f"{support_dir}/{data_folder}"):
    pose_name = os.path.splitext(pose_file)[0].split("_", 1)[1]
    print(pose_name)

yogi_body_hands_03596_Fish_Pose_or_Matsyasana_-d_stageii
yogi_nexus_body_hands_03596_Standing_Split_pose_or_Urdhva_Prasarita_Eka_Padasana_-a_stageii
yogi_nexus_body_hands_03596_Frog_Pose_or_Bhekasana-a_stageii
yogi_body_hands_03596_Garland_Pose_or_Malasana_-a_stageii
yogi_nexus_body_hands_03596_Supta_Baddha_Konasana_-b_stageii
yogi_nexus_body_hands_03596_Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padangusthasana_-a_stageii
yogi_body_hands_03596_Yogic_sleep_pose-a_stageii
yogi_nexus_body_hands_03596_Low_Lunge_pose_or_Anjaneyasana_-a_stageii
yogi_body_hands_03596_Heron_Pose_or_Krounchasana_-c_stageii
yogi_nexus_body_hands_03596_Fish_Pose_or_Matsyasana_-c_stageii
yogi_nexus_body_hands_03596_Feathered_Peacock_Pose_or_Pincha_Mayurasana_-b_stageii
yogi_body_hands_03596_Low_Lunge_pose_or_Anjaneyasana_-a_stageii
yogi_nexus_body_hands_03596_Extended_Revolved_Side_Angle_Pose_or_Utthita_Parsvakonasana_-e_stageii
yogi_body_hands_03596_Eagle_Pose_or_Garudasana_-a_stageii
yogi_body_hands_03596_Rajakapo